In [51]:
import pandas as pd
import carbonpipeline.cli as cli
import cfgrib
import xarray as xr

In [52]:
df = pd.read_csv("predictors.csv")

In [53]:
column_mapping = {
        'CO2_F_MDS': 'CO2',
        'G_F_MDS':   'G',
        'H_F_MDS':   'H',
        'LE_F_MDS':  'LE',
        'LW_IN_F':   'LW_IN',
        'LW_OUT':    'LW_OUT',
        'NETRAD':    'NETRAD',
        'PA_F':      'PA',
        'PPFD_IN':   'PPFD_IN',
        'PPFD_OUT':  'PPFD_OUT',
        'P_F':       'P',
        'RH':        'RH',
        'SW_IN_F':   'SW_IN',
        'SW_OUT':    'SW_OUT',
        'TA_F':      'TA',
        'USTAR':     'USTAR',
        'VPD_F':     'VPD',
        'WD':        'WD',
        'WS_F':      'WS',
        'timestamp': 'timestamp'
    }

In [54]:
filtered_df = cli.filtered_and_renamed_columns(df, column_mapping)
filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'])

In [55]:
filtered_df.iloc[:, :20].loc[30000:30002]

,CO2,G,H,LE,LW_IN,LW_OUT,NETRAD,PA,PPFD_IN,PPFD_OUT,P,RH,SW_IN,SW_OUT,TA,USTAR,VPD,WD,WS,timestamp
30000,-0.30925,-0.166667,-0.178783,NaN,NaN,NaN,-0.187018,0.109958,-0.437500,NaN,NaN,-0.040149,-0.5,NaN,0.079194,-0.478600,-0.427936,0.452266,-0.46872,1997-06-04 00:00:00
30001,-0.30675,-0.166856,-0.167391,NaN,NaN,NaN,-0.183448,0.109777,-0.437504,NaN,NaN,-0.037088,-0.5,NaN,0.075938,-0.490220,-0.430814,0.465692,-0.46683,1997-06-04 01:00:00
30002,-0.30350,-0.167553,-0.161725,NaN,NaN,NaN,-0.183792,0.109669,-0.437500,NaN,NaN,0.143197,-0.5,NaN,0.064722,-0.490415,-0.458755,0.581384,-0.48794,1997-06-04 02:00:00


In [56]:
miss = filtered_df[filtered_df[filtered_df.columns.drop('timestamp')].isnull().any(axis=1)].copy()

In [57]:
miss['year']  = miss['timestamp'].dt.year
miss['month'] = miss['timestamp'].dt.month
miss['day'] = miss['timestamp'].dt.day

In [58]:
import os

datasets = []
directory = "./datasets/"
for name in os.listdir(directory): # source: https://www.geeksforgeeks.org/python-loop-through-folders-and-files-in-directory/
    if name.endswith(".grib"):
        path_to_file = os.path.join(directory, name)
        ds_list = cfgrib.open_datasets(path_to_file, decode_timedelta=True)
        ds = xr.merge(ds_list, compat="override")
        df = ds.to_dataframe()
        datasets.append(df)

combined = pd.concat(datasets, axis=1)

failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed t

In [59]:
short_to_full = {
        '10u': '10m_u_component_of_wind',
        '10v': '10m_v_component_of_wind',
        '2t': '2m_temperature',
        '2d': '2m_dewpoint_temperature',
        'sp': 'surface_pressure',
        'tp': 'total_precipitation',
        'avg_sdlwrf': 'mean_surface_downward_long_wave_radiation_flux',
        'avg_sdlwrfcs': 'mean_surface_downward_long_wave_radiation_flux_clear_sky',
        'avg_sdswrf': 'mean_surface_downward_short_wave_radiation_flux',
        'avg_sdswrfcs': 'mean_surface_downward_short_wave_radiation_flux_clear_sky',
        'ishf': 'instantaneous_surface_sensible_heat_flux',
        'slhf': 'surface_latent_heat_flux',
        'sshf': 'surface_sensible_heat_flux',
        'stl1': 'soil_temperature_level_1',
        'stl2': 'soil_temperature_level_2',
        'stl3': 'soil_temperature_level_3',
        'swvl1': 'volumetric_soil_water_layer_1',
        'swvl2': 'volumetric_soil_water_layer_2',
        'swvl3': 'volumetric_soil_water_layer_3',
        'fal': 'forecast_albedo',
        'zust': 'friction_velocity'
    }

combined = combined.rename(columns=short_to_full)

In [61]:
combined.loc[("1994-01-02", slice(None), slice(None))].head(10)

number  depthBelowLandLayer valid_time  \
latitude longitude step                                                      
75.125   44.875    0 days 01:00:00     0.0                  0.0 1994-01-02   
                   0 days 02:00:00     0.0                  0.0 1994-01-02   
                   0 days 03:00:00     0.0                  0.0 1994-01-02   
                   0 days 04:00:00     0.0                  0.0 1994-01-02   
                   0 days 05:00:00     0.0                  0.0 1994-01-02   
                   0 days 06:00:00     0.0                  0.0 1994-01-02   
                   0 days 07:00:00     0.0                  0.0 1994-01-02   
                   0 days 08:00:00     0.0                  0.0 1994-01-02   
                   0 days 09:00:00     0.0                  0.0 1994-01-02   
                   0 days 10:00:00     0.0                  0.0 1994-01-02   

                                    volumetric_soil_water_layer_1  \
latitude longitude step                                             
75.125   44.875    0 days 01:00:00                       0.000012   
                   0 days 02:00:00                       0.000012   
                   0 days 03:00:00                       0.000012   
                   0 days 04:00:00                       0.000012   
                   0 days 05:00:00                       0.000012   
                   0 days 06:00:00                       0.000012   
                   0 days 07:00:00                       0.000012   
                   0 days 08:00:00                       0.000012   
                   0 days 09:00:00                       0.000012   
                   0 days 10:00:00                       0.000012   

                                    soil_temperature_level_1  \
latitude longitude step                                        
75.125   44.875    0 days 01:00:00                272.961853   
                   0 days 02:00:00                272.961853   
                   0 days 03:00:00                272.961853   
                   0 days 04:00:00                272.961853   
                   0 days 05:00:00                272.961853   
                   0 days 06:00:00                272.961853   
                   0 days 07:00:00                272.961853   
                   0 days 08:00:00                272.961853   
                   0 days 09:00:00                272.961853   
                   0 days 10:00:00                272.961853   

                                    volumetric_soil_water_layer_2  \
latitude longitude step                                             
75.125   44.875    0 days 01:00:00                      -0.000002   
                   0 days 02:00:00                      -0.000002   
                   0 days 03:00:00                      -0.000002   
                   0 days 04:00:00                      -0.000002   
                   0 days 05:00:00                      -0.000002   
                   0 days 06:00:00                      -0.000002   
                   0 days 07:00:00                      -0.000002   
                   0 days 08:00:00                      -0.000002   
                   0 days 09:00:00                      -0.000002   
                   0 days 10:00:00                      -0.000002   

                                    soil_temperature_level_2  \
latitude longitude step                                        
75.125   44.875    0 days 01:00:00                272.892639   
                   0 days 02:00:00                272.892639   
                   0 days 03:00:00                272.892639   
                   0 days 04:00:00                272.892639   
                   0 days 05:00:00                272.892639   
                   0 days 06:00:00                272.892639   
                   0 days 07:00:00                272.892639   
                   0 days 08:00:00                272.892639   
                   0 days 09:00: